# Домашнее задание 2.

- Влияние стандартизации признаков на модель линейной регрессии
- Интерпретация модели линейной регрессии 

В качестве набора данных мы будем пользоваться данными о стоимости домов в King County, Wahington, USA. Перед вами стоит задача предсказания стоимости дома.

## 1. Обучите линейную модель

1. Для обучения используйте только набор данных `kc_house_data_train.csv`, набором данных `kc_house_data_test.csv` вы воспользуетесь в самом конце для тестирования качества.

2. Разбейте выборку `kc_house_data_train.csv` на тренировочную и валидационную в отношении 7:3. Используйте `sklearn.model_selection.train_test_split`. Исключите из рассмотрения столбцы `id`, `zipcode`, `lat`, `long`.

3. Обучите линейную модель `sklearn.linear_model.LinearRegression` (с параметрами по умолчанию) на предсказание столбца `price`.  - Чему равно качество такой модели в терминах Mean Squared Error на тренировочной и валидационной выборке?   
Ответ: MSE train: 47752321019
MSE val: 44600692064.  
- Какие признаки имеют наибольший положительный и отрицательный вес? Насколько это согласуется с вашей интуицией?  
Ответ: Max: waterfront(набережная) 543770 
Min: bedrooms(спальни) -39392,  
что не соответствует интуиции.


In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.options.display.max_columns=30
mse_val = np.zeros(8)
mse_test =np.zeros(8)

In [3]:
X = pd.read_csv('kc_house_data_train.csv', index_col=0)
X_test = pd.read_csv('kc_house_data_test.csv', index_col=0)
y_test = X_test.price
X_test = X_test.drop(["id", 'zipcode', 'lat', 'long', 'price', 'date'], axis = 1)

In [4]:
s = (X.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['date']


In [5]:
y = X.price

X = X.drop(["id", 'zipcode', 'lat', 'long', 'price', 'date'], axis = 1)

In [6]:
X.isnull().sum()

bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
grade            0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
sqft_living15    0
sqft_lot15       0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.7, random_state=5)

In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
lr = model.fit(X_train, y_train)

In [9]:
from sklearn.metrics import mean_squared_error as mse

y_pred = lr.predict(X_train)
y_pred_val = lr.predict(X_val)
y_pred_test = lr.predict(X_test)

print("MSE train:", mse(y_train, y_pred))
mse_val[0] = (mse(y_val, y_pred_val))
print("MSE val:", mse(y_val, y_pred_val))
mse_test[0] = (mse(y_test, y_pred_test))
print("MSE test:", mse(y_test, y_pred_test))

dict_train_1 = {}
for i in range (len(model.coef_)):
    dict_train_1[X_train.columns[i]] = model.coef_[i]
print("Linear coefficients:", dict_train_1)
print("Max:", max(dict_train_1, key = dict_train_1.get), dict_train_1[max(dict_train_1, key = dict_train_1.get)])
print("Min:" , min(dict_train_1, key = dict_train_1.get), dict_train_1[min(dict_train_1, key = dict_train_1.get)])

MSE train: 47752321019.68232
MSE val: 44600692064.98482
MSE test: 46611588405.96249
Linear coefficients: {'bedrooms': -39392.40296475128, 'bathrooms': 60405.47914515011, 'sqft_living': 106.57544556572248, 'sqft_lot': -0.07805306891532382, 'floors': 18325.44624475113, 'waterfront': 543770.5941995028, 'view': 45169.62520782234, 'condition': 14777.408327582256, 'grade': 119568.46159295774, 'sqft_above': 53.36407800074085, 'sqft_basement': 53.21136781673067, 'yr_built': -3720.72629083537, 'yr_renovated': 0.8137902379511388, 'sqft_living15': 22.475426676328425, 'sqft_lot15': -0.43252047401983873}
Max: waterfront 543770.5941995028
Min: bedrooms -39392.40296475128


## 2. Стандартизация признаков

Поскольку в линейной модели мы ищем целевую характеристику ввиде линейной комбинации признаков, то мы бы хотели чтобы величины признаков измерялись в сопоставимых масштабах. Сравните различные подходы к стандартизации признаков:

1. Стандартизация или z-scoring: (x - mean(x)) / std(x). Среднее каждого признака становится равно 0, стандартное отклонение становится равно 1.
2. Нормализация к [0, 1]: (x - min(x)) / (max(x) - min(x))
3. Логарифмирование: log(x + 1). Потенциальным недостатком первых двух подходов является необходимость подсчета статистик на тренировочной выборке и применение их на тестовой.

Обратите внимание что подсчет всех статистик mean, std, min, max должен производиться на тренировочной части данных и применяться к тестовой (потому что на практике у вас нет никакой статистики на этапе предсказание, вы делаете предсказание для конкретного 1ого дома).

Воспользуйтесь методами `sklearn.preprocessing.StandardScaler` и `sklearn.preprocessing.MinMaxScaler`. Обучайте модель линейной регрессии на тренировочной части данных, делайте предсказание и оценивайте качество на валидационной.

4. Какой из подходов дает наилучшее качество (включая отсутствие какой бы то нибыло стандартизации данных)?  
Ответ: лучшее качество получилось при z-scoring и нормализации, но не сильно отличается от того, что было без стандартизации
5. Как меняются веса признаков в линейной модели при применении различных видов стандартизации?  
Ответ: веса признаков меняются, но при этом наибольший положительный и отрицательный вес имеют одинаковые признаки

In [10]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [11]:
z_scaler = StandardScaler()
X_train_scaled = z_scaler.fit_transform(X_train)
X_val_scaled = z_scaler.transform(X_val)
X_test_scaled = z_scaler.transform(X_test)

scaled = model.fit(X_train_scaled, y_train)
y_pred_val_z = scaled.predict(X_val_scaled)
y_pred_test_z = scaled.predict(X_test_scaled)

mse_val[1] = (mse(y_val, y_pred_val_z))
print("MSE val:", mse_val[1])
mse_test[1] = (mse(y_test, y_pred_test_z))
print("MSE test:", mse_test[1])

dict_train_2 = {}
for i in range (len(scaled.coef_)):
    dict_train_2[X_train.columns[i]] = scaled.coef_[i]
print("Linear coefficients:", dict_train_2)
print("Max:", max(dict_train_2, key = dict_train_2.get), dict_train_2[max(dict_train_2, key = dict_train_2.get)])
print("Min:" , min(dict_train_2, key = dict_train_2.get), dict_train_2[min(dict_train_2, key = dict_train_2.get)])

MSE val: 44600692064.987686
MSE test: 46611588405.96165
Linear coefficients: {'bedrooms': -38461.61438468296, 'bathrooms': 46576.524007117114, 'sqft_living': 75015.83554764224, 'sqft_lot': -3143.2019851167256, 'floors': 9800.437588784629, 'waterfront': 45666.8103293275, 'view': 34645.06267486751, 'condition': 9510.361582030606, 'grade': 139561.38023910025, 'sqft_above': 64778.43927520313, 'sqft_basement': 34770.93468400805, 'yr_built': -109828.56407384046, 'yr_renovated': 322.1088818449707, 'sqft_living15': 15371.819906524, 'sqft_lot15': -11996.504140918278}
Max: grade 139561.38023910025
Min: yr_built -109828.56407384046


In [12]:
n_scaler = MinMaxScaler()
X_train_scaled_n = n_scaler.fit_transform(X_train)
X_val_scaled_n = n_scaler.transform(X_val)
X_test_scaled_n = n_scaler.transform(X_test)

norm = model.fit(X_train_scaled_n, y_train)
y_pred_val_n = norm.predict(X_val_scaled_n)
y_pred_test_n = norm.predict(X_test_scaled_n)

mse_val[2] = (mse(y_val, y_pred_val_n))
print("MSE val:", mse_val[2])
mse_test[2] = (mse(y_test, y_pred_test_n))
print("MSE test:", mse_test[2])

dict_train_3 = {}
for i in range (len(norm.coef_)):
    dict_train_3[X_train.columns[i]] = norm.coef_[i]
print("Linear coefficients:", dict_train_3)
print("Max:", max(dict_train_3, key = dict_train_3.get), dict_train_3[max(dict_train_3, key = dict_train_3.get)])
print("Min:" , min(dict_train_3, key = dict_train_3.get), dict_train_3[min(dict_train_3, key = dict_train_3.get)])

MSE val: 44600692064.98767
MSE test: 46611588405.96165
Linear coefficients: {'bedrooms': -1299949.2978340157, 'bathrooms': 483243.83316134685, 'sqft_living': 799555.6332045053, 'sqft_lot': -79891.06258461392, 'floors': 45813.61561185308, 'waterfront': 543770.5941995142, 'view': 180678.50083122845, 'condition': 59109.63331033952, 'grade': 1434821.5391152222, 'sqft_above': 908312.8039939748, 'sqft_basement': 479315.2214802074, 'yr_built': -427883.52344607445, 'yr_renovated': 1639.7873294314195, 'sqft_living15': 130604.70441673714, 'sqft_lot15': -376530.2661312691}
Max: grade 1434821.5391152222
Min: bedrooms -1299949.2978340157


In [13]:
X_train_log = X_train.apply(lambda x: np.log(x+1))
X_val_log = X_val.apply(lambda x: np.log(x+1))
X_test_log = X_test.apply(lambda x: np.log(x+1))

log = model.fit(X_train_log, y_train)
y_pred_val_log = log.predict(X_val_log)
y_pred_test_log = log.predict(X_test_log)

mse_val[3] = (mse(y_val, y_pred_val_log))
print("MSE val:", mse_val[3])
mse_test[3] = (mse(y_test, y_pred_test_log))
print("MSE test:", mse_test[3])

dict_train_4= {}
for i in range (len(log.coef_)):
    dict_train_4[X_train.columns[i]] = log.coef_[i]
print("Linear coefficients:", dict_train_4)
print("Max:", max(dict_train_4, key = dict_train_4.get), dict_train_4[max(dict_train_4, key = dict_train_4.get)])
print("Min:" , min(dict_train_4, key = dict_train_4.get), dict_train_4[min(dict_train_4, key = dict_train_4.get)])

MSE val: 51841036974.26174
MSE test: 54179215275.622505
Linear coefficients: {'bedrooms': -141034.74693485638, 'bathrooms': 169180.50388410335, 'sqft_living': -78614.73611051208, 'sqft_lot': -6481.69347770774, 'floors': 2840.9781507783136, 'waterfront': 904932.7964033512, 'view': 117832.91111080923, 'condition': 68726.5668984285, 'grade': 1175162.9589023707, 'sqft_above': 305317.69108270184, 'sqft_basement': 19338.34587771204, 'yr_built': -7305272.051581271, 'yr_renovated': 2045.0848352860248, 'sqft_living15': 110049.7965941675, 'sqft_lot15': -13037.8799606904}
Max: grade 1175162.9589023707
Min: yr_built -7305272.051581271


## 3. Отбор признаков

1. Воспользуйтесь методом `sklearn.feature_selection.RFE` для отбора 5 наиболее важных признаков. 
- опишите словами в чем заключается метод RFE?  
Ответ: он выделяет наиболее важные признаки и находит их оптимальное количество с помощью ранжирования
- сравните как меняется результат для 3 подходов к стандартизации признаков и для случая без нормировки  
Ответ: с нормировкой результат несильно, но улучшается
- в рамках каждого подхода сравните отобранные 5 признаков с 5 признаками имеющими наибольшую важность для базовой модели (обученной на всех данных). Подготовьте таблицу важности признаков, в которой каждый столбец соответствует какому-то методу нормировки данных, а каждая строка это название признака. Сравните какие признаки оказываются важными в разных сценариях. Прокомментируйте полученный результат.  
Ответ: самыми важными признаками стали класс, набережная и площадь наверху, что соотвествует интуиции. 

|LR|LR-standartized|LR-normalized|LR-log|LR-standartized-RFE|LR-normalized-RFE|LR-log-RFE|

2. Оцените качество каждой из 7 обученных моделей на тестовой части данных `kc_house_data_test.csv` не забудьте применить соответствующие преобразования (стандартизация и пр). Совпадают ли лучшая модель на валидации и на тесте?  
Ответ: лучшей моделью стали LR-normalized и LR-standartized, но качество модели конечно ухудшается на тесте

In [14]:
from sklearn.feature_selection import RFE
rfe = RFE(estimator=model, n_features_to_select=5)

In [15]:
X_test_scaled = z_scaler.transform(X_test)
X_test_scaled_n = n_scaler.transform(X_test)
X_test_log = X_test.apply(lambda x: np.log(x+1))

In [16]:
rfe_1 = rfe.fit(X_train, y_train)
y_pred_rfe = rfe_1.predict(X_val)
y_pred_rfe_test = rfe_1.predict(X_test)

mse_val[4] = (mse(y_val, y_pred_rfe))
print("MSE val without normalization:",mse_val[4])
mse_test[4] = (mse(y_test, y_pred_rfe_test))
print("MSE test without normalization:", mse_test[4])

dict_train_5 = {}
l = 0
for i in range (len(rfe_1.support_)):
    if rfe_1.support_[i] == True:
        dict_train_5[X_train.columns[i]] = rfe_1.estimator_.coef_[l]
        l = l+1

MSE val without normalization: 59826960746.65318
MSE test without normalization: 60988140443.15362


In [17]:
print(rfe_1.support_)
print(X_train.columns)
print(rfe_1.estimator_.coef_)
print(dict_train_5)

[False  True False False False  True  True  True  True False False False
 False False False]
Index(['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'sqft_living15',
       'sqft_lot15'],
      dtype='object')
[ 73826.38785382 558974.08203673  87568.29856773  68756.45457015
 164663.8639791 ]
{'bathrooms': 73826.38785382074, 'waterfront': 558974.082036726, 'view': 87568.29856772996, 'condition': 68756.45457014692, 'grade': 164663.86397910095}


In [18]:
rfe_2 = rfe.fit(X_train_scaled, y_train)
y_pred_rfe_scaled = rfe_2.predict(X_val_scaled)
y_pred_rfe_test_sc = rfe_2.predict(X_test_scaled)

mse_val[5] = (mse(y_val, y_pred_rfe_scaled))
print("MSE val with standartization:", mse_val[5])
mse_test[5] = (mse(y_test, y_pred_rfe_test_sc))
print("MSE test with standartization:", mse_test[5])

dict_train_6 = {}
l = 0
for i in range (len(rfe_2.support_)):
    if rfe_2.support_[i] == True:
        dict_train_6[X_train.columns[i]] = rfe_2.estimator_.coef_[l]
        l = l+1

MSE val with standartization: 47761912049.86946
MSE test with standartization: 48419454301.99658


In [19]:
print(rfe_2.estimator_.coef_)
print(dict_train_6)

[ 167023.34947317   60251.41019383  164969.20471979   -8891.75967626
 -102331.39780581]
{'sqft_living': 167023.34947317277, 'waterfront': 60251.4101938285, 'grade': 164969.2047197853, 'sqft_above': -8891.759676257643, 'yr_built': -102331.39780581126}


In [20]:
rfe_3 = rfe.fit(X_train_scaled_n, y_train)
y_pred_rfe_n = rfe_3.predict(X_val_scaled_n)
y_pred_rfe_test_n = rfe_3.predict(X_test_scaled_n)

mse_val[6] = (mse(y_val, y_pred_rfe_n))
print("MSE val with normalization:", mse_val[6])
mse_test[6] = (mse(y_test, y_pred_rfe_test_n))
print("MSE test with normalization:", mse_test[6])

dict_train_7 = {}
l = 0
for i in range (len(rfe_3.support_)):
    if rfe_3.support_[i] == True:
        dict_train_7[X_train.columns[i]] = rfe_3.estimator_.coef_[l]
        l = l+1

MSE val with normalization: 59241761832.89006
MSE test with normalization: 60634950053.95871


In [21]:
rfe_4 = rfe.fit(X_train_log, y_train)
y_pred_rfe_log = rfe_4.predict(X_val_log)
y_pred_rfe_test_l = rfe_4.predict(X_test_log)

mse_val[7] = (mse(y_val, y_pred_rfe_log))
print("MSE val with log:", mse_val[7] )
mse_test[7] = (mse(y_test, y_pred_rfe_test_l))
print("MSE test with log:", mse_test[7])

dict_train_8 = {}
l = 0
for i in range (15):
    if rfe_4.support_[i] == True:
        dict_train_8[X_train.columns[i]] = rfe_4.estimator_.coef_[l]
        l +=1

MSE val with log: 56025767959.23278
MSE test with log: 57209745331.27951


In [22]:
features = pd.DataFrame({'LR': [sorted(dict_train_1.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)],
                         'LR-standartized': [sorted(dict_train_2.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)],
                         'LR-normalized': [sorted(dict_train_3.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)],
                         'LR-log': [sorted(dict_train_4.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)],
                         'LR-RFE': [sorted(dict_train_5.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)],
                         'LR-standartized-RFE': [sorted(dict_train_6.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)], 
                         'LR-normalized-RFE': [sorted(dict_train_7.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)],
                         'LR-log-RFE':[sorted(dict_train_8.items(), key=lambda x:abs(x[1]), reverse=1)[i][0] for i in range (5)] })
features

,LR,LR-standartized,LR-normalized,LR-log,LR-RFE,LR-standartized-RFE,LR-normalized-RFE,LR-log-RFE
0,waterfront,grade,grade,yr_built,waterfront,sqft_living,bedrooms,yr_built
1,grade,yr_built,bedrooms,grade,grade,grade,grade,grade
2,bathrooms,sqft_living,sqft_above,waterfront,view,yr_built,sqft_living,waterfront
3,view,sqft_above,sqft_living,sqft_above,bathrooms,waterfront,sqft_above,sqft_living
4,bedrooms,bathrooms,waterfront,bathrooms,condition,sqft_above,sqft_basement,bathrooms


In [23]:
final = pd.DataFrame([mse_val, mse_test], index=['mse_val', 'mse_test'], columns = ['LR', 'LR-standartized','LR-normalized','LR-log','LR-RFE','LR-standartized-RFE','LR-normalized-RFE','LR-log-RFE'])
final 

,LR,LR-standartized,LR-normalized,LR-log,LR-RFE,LR-standartized-RFE,LR-normalized-RFE,LR-log-RFE
mse_val,4.460069e+10,4.460069e+10,4.460069e+10,5.184104e+10,5.982696e+10,4.776191e+10,5.924176e+10,5.602577e+10
mse_test,4.661159e+10,4.661159e+10,4.661159e+10,5.417922e+10,6.098814e+10,4.841945e+10,6.063495e+10,5.720975e+10


## Подготовьте репорт о выполненном домашнем задании

jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма [ml-ds-2024-ha2-Surname], например [ml-ds-2023-ha2-Kurmukov] до 12:00 МСК 15.02.2023. Задания отправленные до полуночи Вторника 13.02.2023 получают +1 балл в случае полного выполнения.

Обратите внимание, что нотбук с вашим решением должно быть возможно запустить путем Kernel -> Restart & Run all. И он не должен падать с ошибками. Если он будет падать с ошибками вы можете потерять часть баллов за задания.